In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Age Group Data

In [2]:
df_age_gp = pd.read_csv("zip_code_age_gp.csv", index_col="Unnamed: 0")

In [3]:
df_age_gp.head(2)

,female[0-5],male[0-5],female[6-11],male[6-11],female[12-17],male[12-17],female[18-23],male[18-23],female[24-29],male[24-29],...,female[42-47],male[42-47],female[48-53],male[48-53],female[54-59],male[54-59],female[60-65],male[60-65],female[66-102],male[66-102]
10026,1242,1141,1124,1201,1194,1171,1514,1425,2134,1773,...,1676,1609,1528,1432,1201,1071,911,699,1860,1071
10027,1904,1828,1707,1739,1896,1929,5573,4126,4402,3858,...,2388,2263,2184,1971,1939,1661,1476,1243,3329,1930


## Wealthy data

In [4]:
df_wealthy = pd.read_csv("../data-collect/wealth_data.csv", sep="\t")
df_wealthy = df_wealthy.dropna(axis="index", how='any')
df_wealthy["zip_code"] = df_wealthy["zip_code"].astype('int64')
df_wealthy["count"] = df_wealthy["count"].replace("**", 0).astype('int64')
df_wealthy = df_wealthy[df_wealthy["zip_code"].isin(df_age_gp.index)]  # get the selected zip code 
df_wealthy.head(6)

,zip_code,income_range,count
1,10001,"1-25,000",3760
2,10001,"25,000-50,000",2430
3,10001,"50,000-75,000",1930
4,10001,"75,000-100,000",1340
5,10001,"100,000-200,000",2480
6,10001,"200,000-...",2370


In [5]:
income_range_cols = ["1-25,000", "25,000-50,000", "50,000-75,000", "75,000-100,000", "100,000-200,000", "200,000-..."]
df_wealthy_pivot = df_wealthy.pivot(index="zip_code", columns='income_range', values='count')[income_range_cols]
df_wealthy_pivot.head(2)

income_range,"1-25,000","25,000-50,000","50,000-75,000","75,000-100,000","100,000-200,000","200,000-..."
zip_code,,,,,,
10001,3760,2430,1930,1340,2480,2370
10002,22380,8260,4690,2690,3530,1640


## OpenStreetMap

In [6]:
df_amenities = {osm: pd.read_csv("../openstreetmap/%s_with_codes.csv" % osm, index_col="Unnamed: 0").groupby(by="ZIPCODE").size()
                for osm in ["clinic", "clothes", "gym", "restaurant", "shoes", "stadium"]}

In [7]:
# df_amenities["clinic"][df_amenities["clinic"].index.isin(df_age_gp.index)].to_frame("clinic_count")
# df_amenities["clothes"][df_amenities["clothes"].index.isin(df_age_gp.index)].to_frame("clothes_count")
# df_amenities["gym"][df_amenities["gym"].index.isin(df_age_gp.index)].to_frame("gym_count")
# df_amenities["restaurant"][df_amenities["restaurant"].index.isin(df_age_gp.index)].to_frame("restaurant_count")
# df_amenities["shoes"][df_amenities["shoes"].index.isin(df_age_gp.index)].to_frame("shoes_count")
# df_amenities["stadium"][df_amenities["stadium"].index.isin(df_age_gp.index)].to_frame("stadium_count")

In [8]:
df_all_count = df_age_gp.join(df_wealthy_pivot, how='left') \
    .join(df_amenities["stadium"][df_amenities["stadium"].index.isin(df_age_gp.index)].to_frame("stadium_count"), how="left") \
    .join(df_amenities["shoes"][df_amenities["shoes"].index.isin(df_age_gp.index)].to_frame("shoes_count"), how="left") \
    .join(df_amenities["restaurant"][df_amenities["restaurant"].index.isin(df_age_gp.index)].to_frame("restaurant_count"), how="left") \
    .join(df_amenities["gym"][df_amenities["gym"].index.isin(df_age_gp.index)].to_frame("gym_count"), how="left") \
    .join(df_amenities["clothes"][df_amenities["clothes"].index.isin(df_age_gp.index)].to_frame("clothes_count"), how="left") \
    .join(df_amenities["clinic"][df_amenities["clinic"].index.isin(df_age_gp.index)].to_frame("clinic_count"), how="left")
df_all_count = df_all_count.fillna(0)
df_all_count.head(2)

,female[0-5],male[0-5],female[6-11],male[6-11],female[12-17],male[12-17],female[18-23],male[18-23],female[24-29],male[24-29],...,"50,000-75,000","75,000-100,000","100,000-200,000","200,000-...",stadium_count,shoes_count,restaurant_count,gym_count,clothes_count,clinic_count
10026,1242,1141,1124,1201,1194,1171,1514,1425,2134,1773,...,2520.0,1440.0,1840.0,980.0,0.0,0.0,18.0,0.0,0.0,1.0
10027,1904,1828,1707,1739,1896,1929,5573,4126,4402,3858,...,3490.0,1770.0,2380.0,1280.0,0.0,2.0,25.0,4.0,5.0,1.0


In [13]:
df_all_count.to_csv("zip_code_age_wealth_osm.csv")
df_all_count.to_csv("../zip_code_age_wealth_osm.csv")

## Percentage Data for Clustering

In [38]:
df_age_lg = pd.read_csv("zip_code_age_lg_gp.csv", index_col="Unnamed: 0")

In [47]:
def change_to_percentage(df, population=False):
    df = df.copy()
    for i, row in df.iterrows():
#         print(row, np.sum(row))
        df.loc[i, :] = row / np.sum(row)
        if population: df.loc[i, "population"] = np.sum(row)
    return df

In [49]:
df_age_lg_per = change_to_percentage(df_age_lg, population=True)
df_age_lg_per.head(2)

,female[0-9],male[0-9],female[10-19],male[10-19],female[20-29],male[20-29],female[30-39],male[30-39],female[40-49],male[40-49],female[50-59],male[50-59],female[60-102],male[60-102],population
10026,0.059201,0.056966,0.058436,0.059289,0.094345,0.08111,0.092021,0.078905,0.083287,0.079199,0.064288,0.059407,0.081493,0.052054,34003.0
10027,0.050966,0.050011,0.077194,0.067228,0.131140,0.10853,0.077478,0.069054,0.066776,0.061601,0.055973,0.050430,0.080476,0.053143,59707.0


In [50]:
df_wealthy_pivot_per = change_to_percentage(df_wealthy_pivot)
df_wealthy_pivot_per.head(2)

income_range,"1-25,000","25,000-50,000","50,000-75,000","75,000-100,000","100,000-200,000","200,000-..."
zip_code,,,,,,
10001,0.262753,0.169811,0.134871,0.093641,0.173305,0.165618
10002,0.518176,0.191248,0.108590,0.062283,0.081732,0.037972


In [52]:
df_all_percentage = df_age_lg_per.join(df_wealthy_pivot_per, how='left') \
    .join(df_amenities["stadium"][df_amenities["stadium"].index.isin(df_age_lg_per.index)].to_frame("stadium_count"), how="left") \
    .join(df_amenities["shoes"][df_amenities["shoes"].index.isin(df_age_lg_per.index)].to_frame("shoes_count"), how="left") \
    .join(df_amenities["restaurant"][df_amenities["restaurant"].index.isin(df_age_lg_per.index)].to_frame("restaurant_count"), how="left") \
    .join(df_amenities["gym"][df_amenities["gym"].index.isin(df_age_lg_per.index)].to_frame("gym_count"), how="left") \
    .join(df_amenities["clothes"][df_amenities["clothes"].index.isin(df_age_lg_per.index)].to_frame("clothes_count"), how="left") \
    .join(df_amenities["clinic"][df_amenities["clinic"].index.isin(df_age_lg_per.index)].to_frame("clinic_count"), how="left")
df_all_percentage = df_all_percentage.fillna(0)
df_all_percentage.head(2)

,female[0-9],male[0-9],female[10-19],male[10-19],female[20-29],male[20-29],female[30-39],male[30-39],female[40-49],male[40-49],...,"50,000-75,000","75,000-100,000","100,000-200,000","200,000-...",stadium_count,shoes_count,restaurant_count,gym_count,clothes_count,clinic_count
10026,0.059201,0.056966,0.058436,0.059289,0.094345,0.08111,0.092021,0.078905,0.083287,0.079199,...,0.136957,0.078261,0.100000,0.053261,0.0,0.0,18.0,0.0,0.0,1.0
10027,0.050966,0.050011,0.077194,0.067228,0.131140,0.10853,0.077478,0.069054,0.066776,0.061601,...,0.126863,0.064340,0.086514,0.046529,0.0,2.0,25.0,4.0,5.0,1.0


In [54]:
df_all_percentage.shape

(63, 27)

In [53]:
df_all_percentage.to_csv("zip_code_age_lg_wealth_osm_percentage.csv")
df_all_percentage.to_csv("../zip_code_age_lg_wealth_osm_percentage.csv")

## Wealthy data Weight Average (If Necessary)

In [10]:
for i, row in df_wealthy.iterrows():
    lb, ub = row["income_range"].replace(",", "").split("-")
    if ub != "...":
        lb, ub = int(lb), int(ub)
    else:
        lb, ub = int(lb), int(lb)
    avg = (lb+ub)//2
    df_wealthy.loc[i, "avg_income"] = avg
    
df_wealthy["avg_income"] = df_wealthy["avg_income"].astype('int64')
df_wealthy.head(6)

,zip_code,income_range,count,avg_income
1,10001,"1-25,000",3760,12500
2,10001,"25,000-50,000",2430,37500
3,10001,"50,000-75,000",1930,62500
4,10001,"75,000-100,000",1340,87500
5,10001,"100,000-200,000",2480,150000
6,10001,"200,000-...",2370,200000


In [11]:
def my_groupby_agg():
    for zc in df_age_gp.index:
        rows = df_wealthy.zip_code==zc
        yield {"zip_code": zc,
               "weight_income": (df_wealthy[rows]["count"]@df_wealthy[rows]["avg_income"])/np.sum(df_wealthy[rows]["count"])}
        
# df_weight_income = pd.DataFrame(my_groupby_agg())[["zip_code", "weight_income"]]

In [12]:
pd.merge(pd.DataFrame(my_groupby_agg())[["zip_code", "weight_income"]],
         df_age_gp, right_index=True, left_on="zip_code").head()

/anaconda3/envs/SP/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in long_scalars
  """


,zip_code,weight_income,female[0-5],male[0-5],female[6-11],male[6-11],female[12-17],male[12-17],female[18-23],male[18-23],...,female[42-47],male[42-47],female[48-53],male[48-53],female[54-59],male[54-59],female[60-65],male[60-65],female[66-102],male[66-102]
0,10026,54714.673913,1242,1141,1124,1201,1194,1171,1514,1425,...,1676,1609,1528,1432,1201,1071,911,699,1860,1071
1,10027,50295.347146,1904,1828,1707,1739,1896,1929,5573,4126,...,2388,2263,2184,1971,1939,1661,1476,1243,3329,1930
2,10030,38531.860970,1033,1058,951,982,1150,1165,1412,1258,...,1352,1136,1243,1078,924,833,703,527,1412,938
3,10037,48464.912281,548,504,471,529,590,610,741,664,...,867,647,829,603,782,468,693,449,1916,910
4,10039,38751.840943,968,1035,1006,1030,1086,1074,1195,1151,...,1154,977,1079,838,838,578,737,415,1727,839
